# Bayesian Hyperparameter Tuning

In HW 2 we used grid and random search to find the best hyperparameters for our models. However, these methods are often considered inefficient (as many of you experienced first hand)! In this assingment we will leverage bayesian optimization to perform this task quicker and hopefully more effectively. We will leverage a popular fine tuning library called `optuna` to accomplish this. Thankfully there is a nice blog post on how to do this to help you get started. You can find it here: [optuna tutorial](https://medium.com/@becaye-balde/bayesian-sorcery-for-hyperparameter-optimization-using-optuna-1ee4517e89a). 

Additionally, you can head over to their website to see some additional examples: [https://optuna.org/#code_examples](https://optuna.org/#code_examples)

For this assignment use `optuna` to optimize the hyperparameters of a random forest model to predict the heat capacity dataset from HW 2. Follow the same splitting procedure to ensure that materials aren't mixed between the training and testing sets. Perform the optimization for 10 trials and report the best hyperparameters and the R^2 and MAE on the training and testing sets. How does this compare to your results from HW 2?

In [4]:
#your code goes here
import optuna
import pandas as pd
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import numpy as np
import sys
import os
from CBFV import composition
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score

def objective(trial):
    """
    Define a search space for the hyperparameters `n_estimators` and `max_depth`
    of a random forest model, then train and evaluate it using cross validation.
    """

    # function from https://medium.com/@becaye-balde/bayesian-sorcery-for-hyperparameter-optimization-using-optuna-1ee4517e89a
  
    n_estimators = trial.suggest_int('n_estimators', 1, 10)
    max_depth = int(trial.suggest_float('max_depth', 10, 50, log=True))
    
    model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth)
    
    model.fit(X_train,y_train)

    y_pred = model.predict(X_test)

    r2 = r2_score(y_test, y_pred)
    print('the r2 score is ',r2)
    mae = mean_absolute_error(y_test, y_pred)
    print('the mean absolute error is ',mae)

    return r2


# data is in a csv file stored in c drive to avoid space in my username
filename = 'C:\cp_data_demo.csv'

# read the data from the csv file into a pandas dataframe
df=pd.read_csv(filename)
# change the column names to the column names required by the featurizer
rename_dict = {'CONDITION: Temperature (K)':'Temperature', 'PROPERTY: Heat Capacity (J/mol K)':'target', 'FORMULA':'formula'}
df = df.rename(columns=rename_dict)
# drop NaN values
df = df.dropna()
# reset the index after dropping values
df = df.reset_index(drop=True)
# find the number of unique values in the dataset
nunique=df.nunique()
# get the values of the unique data
unique_vals=pd.Series({c: df[c].unique() for c in df})
# define groups for the k-fold as the unique formula names
groups = df['formula'].unique()

# split data into training and testing sets by splitting the group names
train_groups,test_groups = train_test_split(groups, test_size=0.2, random_state=42)
train_df = df[df['formula'].isin(train_groups)]
test_df = df[df['formula'].isin(test_groups)]

# Featurize the data 
X_train_feats_unscaled, y_train, formulae_train, skipped_train = composition.generate_features(train_df, elem_prop='oliynyk', drop_duplicates=False, extend_features=True, sum_feat=True)
X_test_feats_unscaled, y_test, formulae_train, skipped_train = composition.generate_features(test_df, elem_prop='oliynyk', drop_duplicates=False, extend_features=True, sum_feat=True)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_feats_unscaled)
X_test = scaler.transform(X_test_feats_unscaled)

# Normalize the data
X_train = normalize(X_train)
X_test = normalize(X_test)

# create a study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# get the best trial
trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))
  



Processing Input Data: 100%|██████████| 3589/3589 [00:00<00:00, 8715.73it/s]


	Featurizing Compositions...


Assigning Features...: 100%|██████████| 3589/3589 [00:01<00:00, 3459.04it/s]


	Creating Pandas Objects...


Processing Input Data: 100%|██████████| 981/981 [00:00<00:00, 4626.87it/s]


...

the r2 score is  0.8899922476628883
the mean absolute error is  17.12756019367992


[I 2024-03-30 15:03:01,423] Trial 8 finished with value: 0.861280085822574 and parameters: {'n_estimators': 6, 'max_depth': 12.672700912303323}. Best is trial 0 with value: 0.8908657373826957.


the r2 score is  0.861280085822574
the mean absolute error is  18.148083711124205


[I 2024-03-30 15:03:17,715] Trial 9 finished with value: 0.8808345560594902 and parameters: {'n_estimators': 6, 'max_depth': 14.762518519487127}. Best is trial 0 with value: 0.8908657373826957.


the r2 score is  0.8808345560594902
the mean absolute error is  17.762772274999808
Accuracy: 0.8908657373826957
Best hyperparameters: {'n_estimators': 9, 'max_depth': 47.20814838990367}


Ridge ended with R2 = 0.86 and XGBoost with R2 = 0.9. The optimized Random Forest Regressor performed with an R2 value of 0.89.